In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv('table-14.csv')

# drop the file headers
df = df.drop(range(9))

# reassign first row of data frame (df.iloc[0]) to be the column headers
df.columns = df.iloc[0] 

# reset the index column for the dataframe
df = df[1:].reset_index(drop=True) 

# drop unnecessary columns
df = df.drop(columns=['Financial year end', 'Year End Month']) 

# remove any rows where the Value column is NaN
df = df.dropna(subset=['Value (Ratio)']) 

# Negative values are shown in parentheses. Convert these to standard negative
# number notation to avoid errors.
df['Value (Ratio)'] = df['Value (Ratio)'].str.replace('(','-')
df['Value (Ratio)'] = df['Value (Ratio)'].str.replace(')','')

# remove duplicate rows
df = df.drop_duplicates() 

# convert KFI values to number format:

# Identify non-numeric values before coercion
original_values = df['Value (Ratio)']
numeric_values = pd.to_numeric(original_values, errors='coerce')

# Find rows where conversion to numeric failed (became NaN) and the original was not already NaN or an empty string
conversion_errors = df[numeric_values.isna() & original_values.notna() & (original_values != '')]

if not conversion_errors.empty:
    print('The following original entries in "Value (Ratio)" caused conversion errors and will be set to NaN:')
    display(conversion_errors[['KFI ratio title', 'Value (Ratio)']])
else:
    print('No non-numeric conversion errors found in "Value (Ratio)".')

# Convert 'Value (Ratio)' to numeric, coercing errors
df['Value (Ratio)'] = numeric_values 

# Drop rows where 'Value (Ratio)' became NaN after conversion
df.dropna(subset=['Value (Ratio)'], inplace=True) 

df["Value (Ratio)"] = pd.to_numeric(df["Value (Ratio)"], errors='coerce')

# convert % values to decimals for Tableau / other processing

# Create a boolean mask: rows where column KFI ratio title contains '%'
mask = df["KFI ratio title"].astype(str).str.contains("%", na=False) 
df.loc[mask, "Value (Ratio)"] = df.loc[mask, "Value (Ratio)"] / 100 # Divide column Value (Ratio) by 100 only for those rows

display(df.head())

# save the cleaned file without its index
df.to_csv('processed_table-14.csv', index=False) 

print('Cleaned dataframe saved to processed_table-14.csv')

No non-numeric conversion errors found in "Value (Ratio)".


9,UKPRN,HE provider,Country of HE provider,Region of HE provider,Academic Year,KFI ratio title,Value (Ratio)
0,10007783,The University of Aberdeen,Scotland,Scotland,2015/16,Surplus/(deficit) as a % of total income,-0.020
2,10007783,The University of Aberdeen,Scotland,Scotland,2015/16,Staff costs as a % of total income,0.581
4,10007783,The University of Aberdeen,Scotland,Scotland,2015/16,Premises costs as a % of total costs,0.042
6,10007783,The University of Aberdeen,Scotland,Scotland,2015/16,Unrestricted reserves as a % of total income,1.425
7,10007783,The University of Aberdeen,Scotland,Scotland,2015/16,External borrowing as a % of total income,0.313


Cleaned dataframe saved to processed_table-14.csv
